Are the cities that have dangerous dams on their territories ready to tackle an emergency quickly enough? Do they have dedicated rescue and disaster respose teams?


In [1]:
import geopandas as gpd
import pandas as pd
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

In [2]:
# Data on the existence of disaster response teams, compiled by the IBGE
munic = pd.read_excel("../../data/brazil/munic/Base_MUNIC_2020.xlsx", sheet_name='Gestão de riscos')

In [3]:
# Our columns of interest are:
# MGRD211 – has firefighters?
# MGRD212 – has municipal civil defense?
# MGRD213 – has voluntary municipal civil defense?
# MGRD214 – has municipal guards?
munic = munic[['CodMun', 'Mgrd211', 'Mgrd212', 'Mgrd213', 'Mgrd214']]

munic = munic.rename(columns={
    'CodMun':'code_muni',
    'Mgrd211': 'tem_bombeiros',
    'Mgrd212': 'tem_orgao_defesa_civil',
    'Mgrd213': 'tem_defesa_civil_voluntaria',
    'Mgrd214': 'tem_guarda_municipal',
})

In [4]:
# Id code should have six digits
munic["code_muni"] = munic.code_muni.astype(str).str.slice(0, -1)

In [5]:
# Data on the cities near dangerous dams
cities = gpd.read_feather("../../data/brazil/near-dam-ratios/pop-near-dangerous-dams-by-city.feather")

In [6]:
# Join the datasets
cities = cities.merge(munic, on='code_muni', how='inner')

In [7]:
# People living near a dangerous dam in a city without civil defense
cities[cities.tem_orgao_defesa_civil=="Não"].people_near_dangerous_dams.sum()

122426

In [8]:
# What about the number of cities in the northeast that don't have civil defense bodies?
cities[cities.name_region=='Nordeste'].tem_orgao_defesa_civil.value_counts(normalize=True)

Sim       0.810638
Não       0.182979
Recusa    0.006383
Name: tem_orgao_defesa_civil, dtype: float64

In [9]:
cities[cities.name_region=='Nordeste'].tem_orgao_defesa_civil.value_counts()

Sim       381
Não        86
Recusa      3
Name: tem_orgao_defesa_civil, dtype: int64

In [10]:
# How many people live there?
# People living near a dangerous dam in a city without civil defense
cities[(cities.name_region=='Nordeste') & (cities.tem_orgao_defesa_civil=="Não")].people_near_dangerous_dams.sum()

68648